# Init

## Import

In [ ]:
import os
from itertools import product

import pandas as pd
import numpy as np
from scipy.optimize import minimize_scalar

import seaborn as sns
import matplotlib.pylab as plt

import lsqfit
import gvar as gv

from luescher_nd.database.utilities import read_table

from luescher_nd.zeta.extern.pyzeta import zeta
from luescher_nd.zeta.zeta3d import Zeta3D 

In [ ]:
%load_ext blackcellmagic

## Data

In [ ]:
DATA = os.path.join(os.getcwd(), os.pardir, "data")

In [ ]:
DB_NAME = "db-contact-fv-c-fitted-parity-a-inv.sqlite"

df_cc = read_table(
    os.path.join(DATA, DB_NAME.replace("fv-c", "fv-c-cartesian")),
    dispersion_zeta=False,
    round_digits=1,
    filter_poles=True,
)

df_c = read_table(
    os.path.join(DATA, DB_NAME),
    dispersion_zeta=False,
    round_digits=1,
    filter_poles=True,
)
df_d = read_table(
    os.path.join(DATA, DB_NAME.replace("fv-c", "fv-d")),
    dispersion_zeta=True,
    round_digits=1,
    filter_poles=True,
)
df_cd = read_table(
    os.path.join(DATA, DB_NAME),
    dispersion_zeta=True,
    round_digits=1,
    filter_poles=True,
)

df_c["type"] = "Regular Lüscher"
df_d["type"] = "Dispersion Lüscher"
df_cd["type"] = "RL fit, DL ERE"
df_cc["type"] = "Cartesian Lüscher"

df = df_c.append(df_d, ignore_index=True)
df = df.append(df_cd, ignore_index=True)
df = df.append(df_cc, ignore_index=True)

df.head()


## Find the poles of the zeta function

In [ ]:
poles = np.unique(
    (np.array(np.meshgrid(*[np.arange(10)] * 3)) ** 2).sum(axis=0).flatten()
)
poles = np.append([-10], poles)

zeros = []

L = 1.0
a_inv = -5

for x_start, x_end in zip(poles, np.roll(poles, -1)[:-1]):
    if x_end > 30:
        continue

    res = minimize_scalar(
        lambda x: (a_inv - zeta(x) / np.pi / L) ** 2,
        bounds=(x_start + 0.1, x_end - 0.1),
        method="bounded",
        options={"xatol": 1.0e-12},
    )
    zeros.append(res.x[0])

zeros = np.array(zeros)


In [ ]:
fig, ax = plt.subplots(figsize=(3, 2), dpi=300)

z = Zeta3D(N=200, spherical=False)
#z = zeta

for x_start, x_end in zip(poles, np.roll(poles, -1)[:-1]):
    if x_end > 20:
        break
    x = np.linspace(x_start+0.01, x_end-0.01, 300)
    ax.plot(x, z(x)/np.pi/L, c="blue", ls="--", lw=1)

ax.axhline(a_inv, ls="-", lw=1, color="black", zorder=-1)
for zero in zeros:
    ax.plot(zero, zeta(zero)/np.pi/L, marker="o", ms=2, color="black")
    
ax.set_ylim(-10, 10)
ax.set_xlim(-2, 10)

ax.set_ylabel("$p \cot(\delta_0(p))$ [fm$^{-1}$]", fontsize=8)
ax.set_xlabel(r"$x = \left(\frac{p L}{2 \pi}\right)^2$", fontsize=8)
ax.set_title("$L = 1$ [fm]", fontsize=6)

sns.despine()

plt.show()

# Analysis

## Convergence pattern

In [ ]:
nstep = '1'
dff = df.query("L == @L and nstep == '1' and epsilon < 0.1")

e_inf = zeros * (2 * np.pi / L) ** 2 / 2 / (dff.mass.unique()[0] / 2)

nlevels = dff.nlevel.unique()[:9]

In [ ]:
data = []
fits = {}


def polynomial(epsilon, p):
    out = 0
    for n in range(len(p.keys())):
        out += p[f"x{n}"] * epsilon ** (2 * n)
    return out


class FitArgs:
    def __init__(self, data, fcn, nexp=2):
        self.data = data
        self.fcn = fcn
        self.nexp = nexp
        
    def __call__(self, width):
        prior = {"x0": gv.gvar(self.data[0][0],  np.exp(width))}
        prior.update({f"x{n}": gv.gvar(0, 5 * 2 ** n * np.exp(width)) for n in range(1, nexp)})
        return {"prior": prior, "fcn": self.fcn, "data": self.data, "svdcut": 1.e-13}


num_error = 1.0e-8

for nexp in range(2, 3):
    for key in ["Dispersion Lüscher", "Regular Lüscher", "Cartesian Lüscher"]:
        tf = dff.query("type == @key")
        for nzero, nlevel in enumerate(nlevels):
            epsilon, x = tf.query("nlevel == @nlevel")[["epsilon", "x"]].values.T

            if len(epsilon) < 2:
                continue

            x = gv.gvar(x, [num_error*np.random.uniform(0.5, 1.5) for _ in x])

            fit_args = FitArgs(data=(epsilon, x), fcn=polynomial, nexp=nexp)

            fit = lsqfit.nonlinear_fit(**fit_args(4))

            data.append(
                {
                    "x_fit": fit.p["x0"],
                    "x_luescher": zeros[nzero],
                    "nexp": nexp,
                    "logGBF": fit.logGBF,
                    "chi2/dof": fit.chi2 / fit.dof,
                    "nlevel": nlevel,
                    "type": key,
                    #"p_width": width,
                }
            )

            fits[(nlevel, key, nexp)] = fit

fit_frame = pd.DataFrame(data)

fit_frame["delta_x"] = (fit_frame["x_luescher"] - fit_frame["x_fit"]) / fit_frame[
    "x_luescher"
]

fit_frame.head()


In [ ]:
idx = fit_frame.groupby(["nlevel", "type"])["logGBF"].transform(max) == fit_frame['logGBF']
opt_fit_frame = fit_frame[idx].sort_values(["nlevel", "type"])

print(opt_fit_frame.nexp.unique())

opt_fit_frame.head()

In [ ]:
fig, axs = plt.subplots(figsize=(6, 2), dpi=250, ncols=3, sharex=True, sharey=True)

for ax, key in zip(axs, ["Dispersion Lüscher", "Regular Lüscher", "Cartesian Lüscher"]):
    tf = dff.query("type == @key")
    for nzero, nlevel in enumerate(nlevels):
        x, energy = tf.query("nlevel == @nlevel")[["epsilon", "x"]].values.T

        if len(x) < 2:
            continue

        color = ax.plot(x, energy, ls="None", marker="o", ms=2, label=f"{nlevel}")[
            0
        ].get_color()

        

        best_fit_data = opt_fit_frame.query(
            "type == @key and nlevel == @nlevel"
        ).to_dict("records")[0]
        best_fit = fits[(nlevel, key, best_fit_data["nexp"])]

        x_cont = np.linspace(-0.01, x.max() * 1.5, 100)
        y_fit = best_fit.fcn(x_cont, best_fit.p)

        ax.fill_between(
            x_cont,
            gv.mean(y_fit) - gv.sdev(y_fit),
            gv.mean(y_fit) + gv.sdev(y_fit),
            alpha=0.2,
            color=color,
            lw=0,
        )
        ax.plot(
            x_cont,
            gv.mean(y_fit),
            color=color,
            lw=0.5,
            # , $\chi^2 = {fit.chi2/fit.dof:1.2f}$, $\log(GBF) = {fit.logGBF:1.2f}$",
        )
        ax.plot(0, zeros[nzero], color=color, marker="s", ms=1)

    ax.axvline(0, color="black", zorder=-1, lw=0.5)

    # ax.set_yticks(range(0, 11, 1))
    ax.set_ylim(-1, 12)
    ax.set_title(f"type = {key}", fontsize=8)

    for zero in zeros:
        if zero > 10:
            break
        ax.plot(0, zero, marker="s", color="black", ms=1, zorder=-1)

axs[0].set_ylabel("$x$")
axs[0].set_xlabel("$\epsilon$ [fm]")
axs[-1].legend(
    loc="upper left",
    fontsize=6,
    frameon=False,
    bbox_to_anchor=(1.0, 1.0),
    title="nlevel",
)
sns.despine()

fig.suptitle(
    rf"L = {L}, nstep = {nstep}, $x(\epsilon) = \sum_{{n=0}}^{{n_\exp=3,4}} a_n \epsilon^{{2n}}$",
    y=1.15,
    fontsize=8,
)

plt.show()


## Difference plot

In [ ]:
tmp = opt_fit_frame.copy()

mean_title = "$x_{\mathrm{RL}} - x_{\mathrm{fit}}$ [%]"

tmp[mean_title] = gv.mean(tmp["delta_x"].values * 100)
tmp["dx_sdev"] = gv.sdev(tmp["delta_x"].values * 100)

fig, axs = plt.subplots(dpi=200, figsize=(3, 2), nrows=2, sharex=True)

ax = axs[1]

for key in tmp["type"].unique():
    tf = tmp.query("type == @key and nlevel > 0")
    axs[1].errorbar(
        np.arange(len(tf["nlevel"])),
        tf[mean_title],
        tf["dx_sdev"],
        ls="None",
        elinewidth=1,
        capsize=2,
        label=key
    )
    axs[0].plot(
        np.arange(len(tf["nlevel"])), tf["chi2/dof"], ls="None", marker="o", ms=2
    )

axs[1].set_yscale("log")
axs[0].set_yscale("log")
axs[0].set_ylim(1.e-6, 1.e6)
#axs[1].set_ylim(1.e-3, 3.e0)

ax.legend(
    title=r"Type", fontsize=6, frameon=False, loc="upper left", bbox_to_anchor=(1, 1.8)
)


sns.despine()

axs[1].set_ylabel(mean_title, fontsize=6)
axs[0].set_ylabel(r"$\chi^2/\mathrm{dof}$", fontsize=6)
axs[1].set_xlabel("nlevel")
ax.set_xticks([])

fig.suptitle(
    rf"L = {L}, nstep = {nstep}, $x(\epsilon) = \sum_{{n=0}}^{{n_\exp=3,4}} a_n \epsilon^{{2n}}$",
    y=1.15,
    fontsize=8,
)

plt.show()


In [ ]:
def get_errs(x, z, y_shift = 5):
    x_mean = gv.mean(x)
    y_mean = z(x_mean)/ np.pi + y_shift
    x_lower = x_mean - gv.sdev(x)
    x_upper = x_mean + gv.sdev(x)
    y_lower = z(x_lower)/ np.pi
    y_upper = z(x_upper)/ np.pi
    return x_mean, np.abs(y_mean)#, (y_mean-y_lower, y_upper-y_mean), (x_mean-x_lower, x_upper-x_mean), 
    
    

In [ ]:
fig, ax = plt.subplots(dpi=200, figsize=(3, 2), nrows=1, sharex=True)

markers = {"Cartesian Lüscher": "s", "Dispersion Lüscher": "d", "Regular Lüscher": "o"}

for key in tmp["type"].unique():
    tf = tmp.query("type == @key and x_luescher > 0")
    x, y = get_errs(tf.x_fit.values, z)
    ax.plot(
        x, y,
        ls="--",
        marker=markers[key],
        label=key,
        ms=2,
        lw=.5,
        #elinewidth=.5,
        #capsize=2,
    )
    
#ax.axhline(0, color="black", zorder=-1, label="Input", lw=1)


ax.legend(
    title=r"Type", fontsize=6, frameon=False, loc="upper left", bbox_to_anchor=(1, 1)
)


sns.despine()

ax.set_ylabel(r"$\left|p \cot(\delta_0(p)) + \frac{1}{a_0}\right|$ [fm$^{-1}$]", fontsize=8)
ax.set_xlabel(r"$x = \left(\frac{p L}{2 \pi}\right)^2$", fontsize=8)

fig.suptitle(
    rf"L = {L}, nstep = {nstep}, $x(\epsilon) = \sum_{{n=0}}^{{n_\exp=3,4}} a_n \epsilon^{{2n}}$",
    y=1.15,
    fontsize=8,
)

ax.set_yscale("log")
ax.set_xscale("log")

#ax.set_ylim(1.e-10, 1.e1)
#ax.set_xlim(1.e-3, 1.e1)


plt.show(fig)